## Creating DataFrame

In [15]:
import pandas as pd
music_df = pd.read_csv("spotify_playlist_data.csv")

In [16]:
music_df.head(555)

,Track Name,Artists,Album Name,Album ID,Track ID,Popularity,Release Date,Duration (ms),Explicit,External URLs,...,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo
0,1-800-273-8255,"Logic, Alessia Cara, Khalid",Everybody,1HiN2YXZcc3EjmVZ4WjfBk,5tz69p7tJuGPeMGwNTxYuV,73,2017-05-05,250173,True,https://open.spotify.com/track/5tz69p7tJuGPeMG...,...,0.574,5,-7.788,0,0.0479,0.569000,0.000000,0.1900,0.357,100.023
1,survivin',Bastille,survivin',3aFhgv33grS5tIxAUqwtBC,1lwvJQGhdq6Kyr4BBquf23,49,2020-09-22,173206,False,https://open.spotify.com/track/1lwvJQGhdq6Kyr4...,...,0.780,7,-5.023,1,0.1290,0.250000,0.000000,0.0908,0.680,95.035
2,Gotta Be A Reason,Alec Benjamin,Narrated For You,6jKZplJpy21R5lHaYHHjmZ,0IrtQ83z9jylZaLEJK2eoy,54,2018-11-16,210354,False,https://open.spotify.com/track/0IrtQ83z9jylZaL...,...,0.518,4,-7.582,0,0.0404,0.806000,0.000025,0.1040,0.277,79.997
3,Under Pressure,"Queen, David Bowie",Hot Space,19HKPNkfGggYUnvdKHGTKJ,7rgZTzcjofQJY5kz9FO3OS,49,1982-05-03,246907,False,https://open.spotify.com/track/7rgZTzcjofQJY5k...,...,0.526,2,-12.745,1,0.0522,0.387000,0.000000,0.0879,0.452,113.453
4,The Pretender,Foo Fighters,"Echoes, Silence, Patience & Grace",3ilXDEG0xiajK8AbqboeJz,7x8dCjCr0x6x2lXKujYD34,78,2007-09-25,269373,False,https://open.spotify.com/track/7x8dCjCr0x6x2lX...,...,0.959,9,-4.040,1,0.0431,0.000917,0.000000,0.0280,0.365,172.984
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,In the End,Linkin Park,Hybrid Theory (Bonus Edition),6hPkbAV3ZXpGZBGUvL6jVM,60a0Rd6pjrkxjPbaKzXjfq,87,2000,216880,False,https://open.spotify.com/track/60a0Rd6pjrkxjPb...,...,0.864,3,-5.870,0,0.0584,0.009580,0.000000,0.2090,0.400,105.143
496,Flowers,Miley Cyrus,Endless Summer Vacation,5DvJgsMLbaR1HmAI6VhfcQ,7DSAEUvxU8FajXtRloy8M0,90,2023-08-18,200600,False,https://open.spotify.com/track/7DSAEUvxU8FajXt...,...,0.691,0,-4.775,1,0.0633,0.058400,0.000070,0.0232,0.632,118.048
497,Cuidado,Sophia Treadway,Cuidado,6YCqciyc3cT72TTIddvULV,7miLbD9XU1SOhXxbZPv7dR,40,2024-05-01,179598,False,https://open.spotify.com/track/7miLbD9XU1SOhXx...,...,0.684,5,-6.322,0,0.0798,0.052000,0.000000,0.2510,0.845,94.011
498,Smells Like Teen Spirit,Nirvana,Nevermind (Remastered),2UJcKiJxNryhL050F5Z1Fk,4CeeEOM32jQcH3eN9Q2dGj,82,1991-09-26,301920,False,https://open.spotify.com/track/4CeeEOM32jQcH3e...,...,0.912,1,-4.556,1,0.0564,0.000025,0.000173,0.1060,0.720,116.761


In [17]:
print(music_df.isnull().sum())

Track Name          0
Artists             0
Album Name          0
Album ID            0
Track ID            0
Popularity          0
Release Date        0
Duration (ms)       0
Explicit            0
External URLs       0
Danceability        0
Energy              0
Key                 0
Loudness            0
Mode                0
Speechiness         0
Acousticness        0
Instrumentalness    0
Liveness            0
Valence             0
Tempo               0
dtype: int64


## Tunning Features for Recommendations

In [18]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

data = music_df

In [19]:
scaler = MinMaxScaler()
music_features = music_df[['Danceability', 'Energy', 'Key', 'Loudness', 'Mode', 'Speechiness', 'Acousticness', 'Instrumentalness', 'Liveness', 'Valence', 'Tempo']].values
music_features_scaled = scaler.fit_transform(music_features)

sequence_length = 5  # Define the sequence length for RNN
X, y = [], []
for i in range(len(music_features_scaled) - sequence_length):
    X.append(music_features_scaled[i:i + sequence_length])
    y.append(music_features_scaled[i + sequence_length])
X, y = np.array(X), np.array(y)

model = Sequential()
model.add(LSTM(64, input_shape=(sequence_length, X.shape[2]), return_sequences=True))
model.add(LSTM(64))
model.add(Dense(X.shape[2]))

model.compile(optimizer='adam', loss='mse')
model.summary()

model.fit(X, y, epochs=20, batch_size=32, validation_split=0.2)


### RNN

In [20]:
def rnn_recommendations(input_song_name, num_recommendations=5):
    if input_song_name not in music_df['Track Name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
        return

    input_song_index = music_df[music_df['Track Name'] == input_song_name].index[0]
    input_sequence = music_features_scaled[input_song_index - sequence_length:input_song_index]

    if input_sequence.shape[0] != sequence_length:
        print(f"Not enough data to generate recommendations for '{input_song_name}'.")
        return

    input_sequence = np.expand_dims(input_sequence, axis=0)
    predicted_features = model.predict(input_sequence)

    similarity_scores = cosine_similarity(predicted_features, music_features_scaled)
    similar_song_indices = similarity_scores.argsort()[0][::-1][1:num_recommendations + 1]

    rnn_recommendations = music_df.iloc[similar_song_indices][['Track Name', 'Artists', 'Album Name', 'Release Date', 'Popularity']]
    return rnn_recommendations

## Recommendation Song List

In [21]:
input_song_name = "The Pretender"
rnn_recommendations_list = rnn_recommendations(input_song_name, num_recommendations=10)
print(f"RNN recommended songs for '{input_song_name}':\n")
print(rnn_recommendations_list.to_string(index=False))

Not enough data to generate recommendations for 'The Pretender'.
RNN recommended songs for 'The Pretender':



AttributeError: 'NoneType' object has no attribute 'to_string'

In [ ]:
## Save Recommendation Song List

In [ ]:
import json

# Convert DataFrames to dictionaries
rnn_recommendations_dict = rnn_recommendations.to_dict(orient="records")

# Specify file paths
file_path = "rnn_recommendations.json"

# Write CB recommendations to JSON file
with open(file_path, "w") as json_file:
    json.dump(rnn_recommendations_dict, json_file, indent=4)

print("Recommendations saved to JSON files successfully.")
